## Import necessary libraries

In [18]:
from ipywidgets import widgets
import json
import os

## Set input path to service instance's credential file
Please insert the credentials json file path into "value". Ensure the credentials json file to your AI Core are downloaded and provided in the folder where this notebook is located.
The credential files for each use case resource group is uploaded in the SAP Passvault. You can find the link for your use case here: https://wiki.one.int.sap/wiki/display/IECOE/SAP+AI+Core+Use+Cases
Just download the txt file, place it in this folder and change the extension to .json.

In [29]:
cf_creds_file = widgets.Text(
    value='config/irpa-d26-genaixl-cx-sec-cn-sk.json', # service credentials file path
    placeholder="Path to service instance's credential file",
    description='',
    disabled=False
)

cf_creds_file

Text(value='config/irpa-d26-genaixl-cx-sec-cn-sk.json', placeholder="Path to service instance's credential fil…

Please insert the resource group value. It can be obtained in AI Launchpad under **"Workspaces" -> "AI API Connections" -> "Resource Groups"**
Alternatively your resource group will also be listed on the wiki page from above: https://wiki.one.int.sap/wiki/display/IECOE/SAP+AI+Core+Use+Cases


In [20]:
resource_group = widgets.Text(
    value='default', # resource group change this to the given resource group
    placeholder='Resource group of deployments',
    description='',
    disabled=False   
)

resource_group

Text(value='default', placeholder='Resource group of deployments')

## Set Scenario ID (self-hosted models)
Please insert the Scenario ID. The Scenario ID can be obtained in AI Launchpad under **"ML Operations" -> "Scenarios"**:
As most use cases will be working with the foundation models provided by AICore you can probably just leave the Scenario ID as is.
<img src="imgs/scenarioId.png">
<br/><br/>

In [21]:
scenario_id = widgets.Text(
    value='foundation-models', # scenario ID
    placeholder='Scenario ID',
    description='',
    disabled=False
)

scenario_id

Text(value='foundation-models', placeholder='Scenario ID')

## Set Config Names (self-hosted models)
Please insert the Configuration Names value. The Configuration Names can be obtained in AI Launchpad under **"ML Operations" -> "Configurations".**


To add multiple configuration names use the prefix with **\*** in the end (e.g. **"command-*"** would include the configurations **"command-config"** and **"command-nightly-config"**)

In [22]:
config_names = widgets.Text(
    value='fine-tuned-*', # config names value
    placeholder='Configuration names',
    description='',
    disabled=False
)

config_names

Text(value='fine-tuned-*', placeholder='Configuration names')

## Set prediction URL suffix (self-hosted models)

In [7]:
prediction_url_suffix = widgets.Text(
    value='/v1/chat/completions',
    placeholder='',
    description='',
    disabled=False
)

prediction_url_suffix

Text(value='/v1/chat/completions', placeholder='')

## Create environment variables from service instance credentials file
Execute this cell to set the necessary env variables

In [30]:
with open(cf_creds_file.value) as cf:
    credCF = json.load(cf)

base_url=credCF["serviceurls"]["AI_API_URL"] + "/v2", # The present SAP AI Core API version is 2
auth_url= credCF["url"] + "/oauth/token", # Suffix to add
client_id=credCF['clientid'],
client_secret=credCF['clientsecret']

os.environ['AICORE_LLM_AUTH_URL'] = auth_url[0]
os.environ['AICORE_LLM_CLIENT_ID'] = client_id[0]
os.environ['AICORE_LLM_CLIENT_SECRET'] = client_secret
os.environ['AICORE_LLM_API_BASE'] = base_url[0]
os.environ['AICORE_LLM_RESOURCE_GROUP'] = resource_group.value
os.environ['LLM_COMMONS_PROXY'] = 'aicore'

## Import llm-commons libraries

In [31]:
from llm_commons.proxy.identity import AICoreProxyClient
from llm_commons.langchain.proxy import ChatOpenAI

## Create AI Core Proxy client
Execute this cell to create the AI Core Proxy client.

In [25]:
aic_proxy_client = AICoreProxyClient()


In [32]:
from llm_commons.proxy.base import set_proxy_version
set_proxy_version('aicore') # for an AI Core proxy

## Get all deployments

In [26]:
aic_proxy_client.get_deployments()

[Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dc8e3c266c8881f0', config_id='58a5ffea-1afe-44b4-b6b9-ca749776e58e', config_name='tiiuae--falcon-40b-instruct-config', deployment_id='dc8e3c266c8881f0', model_name='tiiuae--falcon-40b-instruct', created_at=datetime.datetime(2024, 1, 25, 12, 53, 26, tzinfo=datetime.timezone.utc), additonal_parameters={'executable_id': 'aicore-opensource', 'model_version': 'null'}, custom_prediction_suffix=None),
 Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dd88f66091ca1982', config_id='a99e09fb-7e88-4a8d-abe8-af7f8b146cb8', config_name='text-embedding-ada-002-config', deployment_id='dd88f66091ca1982', model_name='text-embedding-ada-002', created_at=datetime.datetime(2024, 1, 25, 12, 16, 20, tzinfo=datetime.timezone.utc), additonal_parameters={'executable_id': 'azure-openai', 'model_version': '2'}, custom_prediction_suffix=None),
 Deployment(url='https

## Make the experimental deployments known to llm-commons (self-hosted models)
Execute this cell to add self-hosted models to the AI Core client. 

Please skip this step if you just want to access the foundation models deployed on AI Core. They should already be visible and accessible.

In [12]:
# make the deployments of experimental models known to llm-commons
aic_proxy_client.add_foundation_model_scenario(
    scenario_id=scenario_id.value,
    config_names=config_names.value,
    prediction_url_suffix=prediction_url_suffix.value
)

# get all deployments including self-hosted models
aic_proxy_client.get_deployments()

NameError: name 'scenario_id' is not defined

## Predict with ChatOpenAI library
Use the proxy model name from the above ```.get_deployments()``` response (**"model_name"**).

In [34]:
chat = ChatOpenAI(proxy_model_name='gpt-4')
chat.predict('新股发行时各参与方的分成比例')

'新股发行时的分成比例因情况而异，主要取决于公司、投资者、承销商和其他参与方之间的协议。一般来说，以下是一个大概的分配：\n\n1. 公司：公司是新股发行的主要受益者。该公司通过发行新股筹集资金，用于扩大业务、偿还债务或进行其他投资。公司将获得新股发行收入的绝大部分，比例可能在90%到95%之间。\n\n2. 投资者：投资者购买新股，通常会期待随着公司的增长和成功，股票价值会增加。他们没有直接从新股发行中获得分成，而是通过股票的升值和股息收益。\n\n3. 承销商：承销商通常会帮助公司进行新股发行，并在过程中提供各种服务，包括市场研究、定价和分销股票。作为报酬，承销商通常会收取新股发行收入的一部分作为佣金，比例可能在5%到10%之间。\n\n4. 其他参与方：这可能包括法律顾问、会计师、咨询公司等。他们在新股发行过程中提供专业服务，并按照协议收取费用。这通常以固定费用的形式，而非新股发行收入的分成。\n\n请注意，以上比例只是一般的情况，实际情况可能会根据具体的协议、市场条件和其他因素有所不同。'

In [13]:
chat = ChatOpenAI(proxy_model_name='gpt-4-32k')
chat.predict('i need to setup some goals for a compliance team')

"1. Improve Compliance Awareness: Set up regular training sessions and workshops to increase the team's knowledge and understanding of compliance-related issues. \n\n2. Develop a Comprehensive Compliance Program: Aim to develop an all-inclusive compliance program that covers all aspects of the organization's operations.\n\n3. Regular Compliance Audits: Conduct regular compliance audits to identify any potential issues or non-compliance areas that need rectifying.\n\n4. Update Compliance Policies: Ensure all compliance policies are up-to-date, reflecting any changes in industry standards or regulations.\n\n5. Enhance Reporting Procedures: Implement clear and effective reporting procedures for compliance issues. \n\n6. Reduce Non-Compliance Risks: Develop strategies to minimize the risks of non-compliance, such as penalties, fines, and reputational damage.\n\n7. Improve Compliance Communication: Improve communication regarding compliance issues both within the team and across the organiz

In [14]:
chat = ChatOpenAI(proxy_model_name='gpt-35-turbo')
chat.predict('What is your name?')

'I am a language model AI created by OpenAI and do not have a personal name. You can refer to me as OpenAI or as a language model AI. How can I assist you today?'

In [15]:
chat = ChatOpenAI(proxy_model_name='gpt-35-turbo-16k')
chat.predict('What is your name?')

"I am an AI language model developed by OpenAI and I don't have a personal name. You can simply refer to me as OpenAI Assistant. How can I assist you today?"

## Predict with langchain
Use the proxy model name from the above ```.get_deployments()``` response (**"model_name"**).

In [18]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(proxy_model_name='gpt-4-32k')
template = "Question: {question}\nAnswer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=['question'])
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
print(llm_chain.run("What is the world's best company in shanghai?"))



> Entering new LLMChain chain...
Prompt after formatting:
Question: What is the world's best company in shanghai?
Answer: Let's think step by step.

> Finished chain.
Determining the "best" company can depend on various factors such as revenue, employee satisfaction, impact on the industry, etc. According to Forbes' list of top companies in the world, China's ICBC bank, headquartered in Shanghai, ranks high. Moreover, Shanghai is home to many other successful companies such as Bright Food Group and SAIC Motor. However, it's subjective and can vary based on the parameters you are considering.


NameError: name 'date' is not defined

In [4]:
import socket

# 要测试的 IPv6 地址列表
ipv6_addresses = [
    "2606:4700::6810:323",
    "2606:4700::6810:1922",
    "2606:4700::6810:1f22",
    "2606:4700::6810:1c22",
    "2606:4700::6810:23",
    "2606:4700::6810:1a22",
    "2606:4700::6810:1b22",
    "2606:4700::6810:1d22",
    "2606:4700::6810:1e22",
    "2606:4700::6810:223",
    "2606:4700::6810:1822",
    "2606:4700::6810:123",
]

# 逐个测试 IPv6 地址是否可以访问
for ipv6_address in ipv6_addresses:
    try:
        # 使用 socket 模块连接到 registry.yarnpkg.com，并设置超时时间为 5 秒
        sock = socket.socket(socket.AF_INET6, socket.SOCK_STREAM)
        sock.settimeout(5)
        sock.connect((ipv6_address, 443))
        sock.close()

        # 如果连接成功，则打印出 IPv6 地址并提示可以访问
        print(f"{ipv6_address} 可以访问")
    except socket.error:
        # 如果连接失败，则打印出 IPv6 地址并提示无法访问
        print(f"{ipv6_address} 无法访问")



2606:4700::6810:323 无法访问
2606:4700::6810:1922 无法访问
2606:4700::6810:1f22 无法访问
2606:4700::6810:1c22 无法访问
2606:4700::6810:23 无法访问
2606:4700::6810:1a22 无法访问
2606:4700::6810:1b22 无法访问
2606:4700::6810:1d22 无法访问
2606:4700::6810:1e22 无法访问
2606:4700::6810:223 无法访问
2606:4700::6810:1822 无法访问
2606:4700::6810:123 无法访问


In [3]:
import requests

domain = "registry.yarnpkg.com"
ip_list= [
    "104.16.27.34",
    "104.16.3.35",
    "104.16.28.34",
    "104.16.25.34",
    "104.16.0.35",
    "104.16.2.35",
    "104.16.1.35",
    "104.16.24.34",
    "104.16.30.34",
    "104.16.26.34",
    "104.16.29.34",
    "104.16.31.34"
]
reachable_ips = []

# 循环遍历每个IP地址进行HTTPS连接测试
for ip in ip_list:
    url = f"https://{ip}"
    print(f"Testing HTTPS reachability of {url}...")
    
    try:
        # 发送一个GET请求，默认超时时间通常为2-3秒左右
        response = requests.get(url, timeout=5, verify=False)  # 注意：verify=False忽略了SSL证书验证，生产环境中请确保正确处理证书验证

        # 检查响应状态码
        if response.status_code == 403:
            print(f"Host {ip} is reachable over HTTPS (HTTP status code: {response.status_code}).")
            reachable_ips.append(ip)
        else:
            print(f"Host {ip} is not responding properly to HTTPS requests (HTTP status code: {response.status_code}).")

    except requests.exceptions.RequestException as e:
        # 如果发生任何网络错误（如Timeout、ConnectionError等）
        print(f"Host {ip} is unreachable over HTTPS due to error: {str(e)}.")
print("\nReachable IP addresses:")
for reachable_ip in reachable_ips:
    print(reachable_ip)
# 注意：在真实场景下，如果目标服务端口不是443（默认HTTPS端口），需要在URL中指定端口号。

Testing HTTPS reachability of https://104.16.27.34...
Host 104.16.27.34 is unreachable over HTTPS due to error: HTTPSConnectionPool(host='104.16.27.34', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000022A43FEE050>, 'Connection to 104.16.27.34 timed out. (connect timeout=5)')).
Testing HTTPS reachability of https://104.16.3.35...


c:\python\3.11\Lib\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '104.16.3.35'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Host 104.16.3.35 is reachable over HTTPS (HTTP status code: 403).
Testing HTTPS reachability of https://104.16.28.34...


c:\python\3.11\Lib\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '104.16.28.34'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Host 104.16.28.34 is reachable over HTTPS (HTTP status code: 403).
Testing HTTPS reachability of https://104.16.25.34...
Host 104.16.25.34 is unreachable over HTTPS due to error: HTTPSConnectionPool(host='104.16.25.34', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000022A43FCCB50>, 'Connection to 104.16.25.34 timed out. (connect timeout=5)')).
Testing HTTPS reachability of https://104.16.0.35...
Host 104.16.0.35 is unreachable over HTTPS due to error: HTTPSConnectionPool(host='104.16.0.35', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000022A43FE7010>, 'Connection to 104.16.0.35 timed out. (connect timeout=5)')).
Testing HTTPS reachability of https://104.16.2.35...
Host 104.16.2.35 is unreachable over HTTPS due to error: HTTPSConnectionPool(host='104.16.2.35', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutEr

c:\python\3.11\Lib\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '104.16.30.34'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Host 104.16.30.34 is reachable over HTTPS (HTTP status code: 403).
Testing HTTPS reachability of https://104.16.26.34...
Host 104.16.26.34 is unreachable over HTTPS due to error: HTTPSConnectionPool(host='104.16.26.34', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000022A43FEE250>, 'Connection to 104.16.26.34 timed out. (connect timeout=5)')).
Testing HTTPS reachability of https://104.16.29.34...


c:\python\3.11\Lib\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '104.16.29.34'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Host 104.16.29.34 is reachable over HTTPS (HTTP status code: 403).
Testing HTTPS reachability of https://104.16.31.34...
Host 104.16.31.34 is reachable over HTTPS (HTTP status code: 403).

Reachable IP addresses:
104.16.3.35
104.16.28.34
104.16.30.34
104.16.29.34
104.16.31.34


c:\python\3.11\Lib\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '104.16.31.34'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
